In [1]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input
from collections import OrderedDict

In [2]:
image_folder = "all_images"
image_list = []
label_list = []
up_endplates = []
low_endplates = []
disc_herniation = []
disc_narrowing = []
disc_bulging = []
spondylolisthesis = []

df = pd.read_csv('AggregatePTData.csv')

In [3]:
for image_filename in os.listdir(image_folder):
    if image_filename.endswith(".png"):
        image = Image.open(os.path.join(image_folder, image_filename))
        # Resize image to 224x224 pixels
        image = image.resize((224,224))
        # Convert image to RGB
        image = image.convert("RGB")
        # Convert the image to numpy array
        image_array = np.array(image)
        # Normalize array
        image_array = preprocess_input(image_array)
        image_list.append(image_array)
        
        row = df[df['file_name'] == image_filename]
        up = row.iloc[0]['UP endplate']
        up_endplates.append(up)
        low = row.iloc[0]['LOW endplate']
        low_endplates.append(low)
        herniation = row.iloc[0]['Disc herniation']
        disc_herniation.append(herniation)
        narrowing = row.iloc[0]['Disc narrowing']
        disc_narrowing.append(narrowing)
        bulging = row.iloc[0]['Disc bulging']
        disc_bulging.append(bulging)
        spondy = row.iloc[0]['Spondylolisthesis']
        spondylolisthesis.append(spondy)

image_list_np = np.array(image_list)
up_endplates_np = np.array(up_endplates)
up_endplates_np = np.round(up_endplates_np).astype(int)
low_endplates_np = np.array(low_endplates)
low_endplates_np = np.round(low_endplates_np).astype(int)
disc_herniation_np = np.array(disc_herniation)
disc_herniation_np = np.round(disc_herniation_np).astype(int)
disc_narrowing_np = np.array(disc_narrowing)
disc_narrowing_np = np.round(disc_narrowing_np).astype(int)
disc_bulging_np = np.array(disc_bulging)
disc_bulging_np = np.round(disc_bulging_np).astype(int)
spondylolisthesis_np = np.array(spondylolisthesis)
spondylolisthesis_np = np.round(spondylolisthesis_np).astype(int)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer
up_predictions = Dense(1, activation='sigmoid', name='up_output')(x)
low_predictions = Dense(1, activation='sigmoid', name='low_output')(x)
herniation_predictions = Dense(1, activation='sigmoid', name='herniation_output')(x)
narrowing_predictions = Dense(1, activation='sigmoid', name='narrowing_output')(x)
bulging_predictions = Dense(1, activation='sigmoid', name='bulging_output')(x)
spondy_predictions = Dense(1, activation='sigmoid', name='spondy_output')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=[up_predictions, low_predictions, herniation_predictions, narrowing_predictions, bulging_predictions, spondy_predictions])
model.compile(optimizer='adam', loss={
        'up_output': 'binary_crossentropy',
        'low_output': 'binary_crossentropy',
        'herniation_output': 'binary_crossentropy',
        'narrowing_output': 'binary_crossentropy',
        'bulging_output': 'binary_crossentropy',
        'spondy_output': 'binary_crossentropy'
    },          
    metrics={
        'up_output': 'accuracy',
        'low_output': 'accuracy',
        'herniation_output': 'accuracy',
        'narrowing_output': 'accuracy',
        'bulging_output': 'accuracy',
        'spondy_output': 'accuracy'
    }
)

In [6]:
history = model.fit(
    image_list_np,            # Input images
    [up_endplates_np, low_endplates_np, disc_herniation_np, disc_narrowing_np, disc_bulging_np, spondylolisthesis_np],
    batch_size=32,
    epochs=10,
    validation_split=0.2,     # Split 20% of data for validation
    verbose=1                 # Set to 1 to see training progress
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - bulging_output_accuracy: 0.5742 - bulging_output_loss: 0.9393 - herniation_output_accuracy: 1.0000 - herniation_output_loss: 0.0761 - loss: 3.7713 - low_output_accuracy: 0.5573 - low_output_loss: 0.7027 - narrowing_output_accuracy: 0.6501 - narrowing_output_loss: 0.6723 - spondy_output_accuracy: 0.7060 - spondy_output_loss: 0.4533 - up_output_accuracy: 0.5313 - up_output_loss: 0.9276 - val_bulging_output_accuracy: 0.7671 - val_bulging_output_loss: 0.5553 - val_herniation_output_accuracy: 1.0000 - val_herniation_output_loss: 2.2325e-06 - val_loss: 2.2982 - val_low_output_accuracy: 0.6849 - val_low_output_loss: 0.7382 - val_narrowing_output_accuracy: 0.8493 - val_narrowing_output_loss: 0.3757 - val_spondy_output_accuracy: 1.0000 - val_spondy_output_loss: 8.2564e-15 - val_up_output_accuracy: 0.5479 - val_up_output_loss: 0.6768
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - bulging_output_accuracy: 0.6718 - bulging_output_loss: 0.5790 - 

In [7]:
test_image_path = 'all_images/10_t2.png'

# Load, resize, and preprocess the test image
img = Image.open(test_image_path).convert('RGB')
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)

# Preprocess the image for ResNet50
img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img_array)
img_preprocessed = np.expand_dims(img_preprocessed, axis=0)  # Add batch dimension

# Run the prediction
predicted_probabilities = model.predict(img_preprocessed)
print(f"Probability of Up-Endplate: {predicted_probabilities[0][0][0]}")
print(f"Probability of Low-Endplate: {predicted_probabilities[1][0][0]}")
print(f"Probability of Disc-Herniation: {predicted_probabilities[2][0][0]}")
print(f"Probability of Disc-Narrowing: {predicted_probabilities[3][0][0]}")
print(f"Probability of Disc-Bulging: {predicted_probabilities[4][0][0]}")
print(f"Probability of Spondylolisthesis: {predicted_probabilities[5][0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Probability of Up-Endplate: 0.9834853410720825
Probability of Low-Endplate: 0.9761836528778076
Probability of Disc-Herniation: 2.4905928430740687e-09
Probability of Disc-Narrowing: 0.9582281112670898
Probability of Disc-Bulging: 0.9859817028045654
Probability of Spondylolisthesis: 7.191185774344098e-20


In [ ]:
model.summary()

In [8]:
model.save('binaryPredictions.keras')